In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

from functools import partial
import tensorflow as tf

In [ ]:
import tensorflow as tf
from tensorflow.python.ops import nn
from tensorflow.python.keras import activations, regularizers, initializers, constraints, engine
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.keras.layers import Layer, deserialize, Conv1D
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops


__all__ = ['KernelConv2D']


class KernelConv2D(Layer):
    """2D convolution layer (e.g. spatial convolution over images).
    This layer creates a convolution kernel that is convolved
    with the layer input to produce a tensor of
    outputs. If `use_bias` is True,
    a bias vector is created and added to the outputs. Finally, if
    `activation` is not `None`, it is applied to the outputs as well.
    When using this layer as the first layer in a model,
    provide the keyword argument `input_shape`
    (tuple of integers, does not include the sample axis),
    e.g. `input_shape=(128, 128, 3)` for 128x128 RGB pictures
    in `data_format="channels_last"`.
    # Arguments
        filters: Integer, the dimensionality of the output space
            (i.e. the number of output filters in the convolution).
        kernel_size: An integer or tuple/list of 2 integers, specifying the
            height and width of the 2D convolution window.
            Can be a single integer to specify the same value for
            all spatial dimensions.
        kernel_function: A layer takes the columnized feature and the kernel as its inputs.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution
            along the height and width.
            Can be a single integer to specify the same value for
            all spatial dimensions.
            Specifying any stride value != 1 is incompatible with specifying
            any `dilation_rate` value != 1.
        padding: one of `"valid"` or `"same"` (case-insensitive).
            Note that `"same"` is slightly inconsistent across backends with
            `strides` != 1, as described
            [here](https://github.com/keras-team/keras/pull/9473#issuecomment-372166860)
        data_format: A string,
            one of `"channels_last"` or `"channels_first"`.
            The ordering of the dimensions in the inputs.
            `"channels_last"` corresponds to inputs with shape
            `(batch, height, width, channels)` while `"channels_first"`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
        dilation_rate: an integer or tuple/list of 2 integers, specifying
            the dilation rate to use for dilated convolution.
            Can be a single integer to specify the same value for
            all spatial dimensions.
            Currently, specifying any `dilation_rate` value != 1 is
            incompatible with specifying any stride value != 1.
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix.
        bias_initializer: Initializer for the bias vector.
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix.
        bias_regularizer: Regularizer function applied to the bias vector.
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
        kernel_constraint: Constraint function applied to the kernel matrix.
        bias_constraint: Constraint function applied to the bias vector.
    # Input shape
        4D tensor with shape:
        `(batch, channels, rows, cols)`
        if `data_format` is `"channels_first"`
        or 4D tensor with shape:
        `(batch, rows, cols, channels)`
        if `data_format` is `"channels_last"`.
    # Output shape
        4D tensor with shape:
        `(batch, filters, new_rows, new_cols)`
        if `data_format` is `"channels_first"`
        or 4D tensor with shape:
        `(batch, new_rows, new_cols, filters)`
        if `data_format` is `"channels_last"`.
        `rows` and `cols` values might have changed due to padding.
    """
    def __init__(self, filters,
                 kernel_size,
                 kernel_function,
                 strides=1,
                 padding='valid',
                 data_format=None,
                 dilation_rate=1,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(KernelConv2D, self).__init__(**kwargs)
        self.rank = 1
        self.filters = filters
        self.kernel_function = kernel_function
        self.kernel_size = conv_utils.normalize_tuple(kernel_size, self.rank, 'kernel_size')
        self.strides = conv_utils.normalize_tuple(strides, self.rank, 'strides')
        self.padding = conv_utils.normalize_padding(padding)
        self.data_format = conv_utils.normalize_data_format(data_format)
        self.dilation_rate = conv_utils.normalize_tuple(dilation_rate, self.rank, 'dilation_rate')
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = engine.base_layer.InputSpec(ndim=self.rank + 2)

        self.kernel = self.bias = None

    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape.dims[channel_axis].value is None:
            raise ValueError('The channel dimension of the inputs should be defined. Found `None`.')
        input_dim = int(input_shape[channel_axis])
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.kernel = self.add_weight(
            shape=kernel_shape,
            initializer=self.kernel_initializer,
            name='kernel',
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        if self.use_bias:
            self.bias = self.add_weight(
                shape=(self.filters,),
                initializer=self.bias_initializer,
                name='bias',
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
            )
        else:
            self.bias = None

        self.input_spec = engine.base_layer.InputSpec(
            ndim=self.rank + 2,
            axes={channel_axis: input_dim},
        )
        self.kernel_function.build([input_shape, kernel_shape])
        super(KernelConv2D, self).build(input_shape)

    def _compute_causal_padding(self):
      left_pad = self.dilation_rate[0] * (self.kernel_size[0] - 1)
      if self.data_format == 'channels_last':
        causal_padding = [[0, 0], [left_pad, 0], [0, 0]]
      else:
        causal_padding = [[0, 0], [0, 0], [left_pad, 0]]
      return causal_padding

    def call(self, inputs):
        data_format = conv_utils.convert_data_format(self.data_format, self.rank + 2)
        inputs, tf_data_format = K._preprocess_conv2d_input(inputs, self.data_format)

        '''inputs = tf.compat.v1.extract_image_patches(
            inputs,
            ksizes=(1,) + K.int_shape(self.kernel)[:2] + (1,),
            strides=(1,) + self.strides + (1,) + (1,) ,
            rates=(1,) + self.dilation_rate + (1,) + (1,),
            padding=self.padding.upper(),
        )'''
        inputs = array_ops.pad(inputs, self._compute_causal_padding())

        kernel = K.reshape(self.kernel, (-1, self.filters))
        outputs = self.kernel_function([inputs, kernel])

        if self.data_format == 'channels_first':
            outputs = K.permute_dimensions(outputs, (0, 1, 2))

        if self.use_bias:
            outputs = nn.bias_add(outputs, self.bias, data_format=data_format)

        if self.activation is not None:
            outputs = self.activation(outputs)
        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            space = input_shape[1:-1]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i],
                    dilation=self.dilation_rate[i])
                new_space.append(new_dim)
            return (input_shape[0],) + tuple(new_space) + (self.filters,)
        if self.data_format == 'channels_first':
            space = input_shape[2:]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i],
                    dilation=self.dilation_rate[i])
                new_space.append(new_dim)
            return (input_shape[0], self.filters) + tuple(new_space)

    def get_config(self):
        config = {
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'kernel_function': {
                'class_name': self.kernel_function.__class__.__name__,
                'config': self.kernel_function.get_config(),
            },
            'strides': self.strides,
            'padding': self.padding,
            'data_format': self.data_format,
            'dilation_rate': self.dilation_rate,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(KernelConv2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config, custom_objects=None):
        config['kernel_function'] = deserialize(
            config.pop('kernel_function'),
            custom_objects=custom_objects,
        )
        return cls(**config)

In [ ]:
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Flatten, Dense,AveragePooling1D,GRU,Convolution1D, MaxPooling1D, AveragePooling1D,Embedding,Input, Dense, Dropout, Flatten

In [ ]:
class GaussianKernel(Layer):

    def __init__(self, gamma, **kwargs):
        super(GaussianKernel, self).__init__(**kwargs)
        self.gamma = gamma

    def call(self, inputs, **kwargs):
        x, kernel = K.expand_dims(inputs[0], axis=-1), inputs[1]
        return K.exp(-self.gamma * K.sum(K.square(x - kernel), axis=-2))

    def get_config(self):
        config = {'gamma': self.gamma}
        base_config = super(GaussianKernel, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class PolynomialKernel(Layer):

    def __init__(self, p,
                 c=0.0,
                 trainable_c=False,
                 initializer='zeros',
                 regularizer=None,
                 constraint=None,
                 **kwargs):
        super(PolynomialKernel, self).__init__(**kwargs)
        self.p = p
        self.c = c
        self.oc = c
        self.trainable_c = trainable_c
        self.initializer = initializers.get(initializer)
        self.regularizer = regularizers.get(regularizer)
        self.constraint = constraints.get(constraint)

    def build(self, input_shape):
        if self.trainable_c:
            self.c = self.add_weight(
                shape=(),
                initializer=self.initializer,
                regularizer=self.regularizer,
                constraint=self.constraint,
                name='{}_c'.format(self.name),
            )
        super(PolynomialKernel, self).build(input_shape)

    def call(self, inputs, **kwargs):
        return (K.dot(inputs[0], inputs[1]) + self.c) ** self.p

    def get_config(self):
        config = {
            'p': self.p,
            'c': self.oc,
            'trainable_c': self.trainable_c,
            'initializer': initializers.serialize(self.initializer),
            'regularizer': regularizers.serialize(self.regularizer),
            'constraint': initializers.serialize(self.constraint),
        }
        base_config = super(PolynomialKernel, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
import pandas as pd
dataset=pd.read_csv('New_Created_Data/100_tweets_per_user_new.csv')
 

In [ ]:
k=0
X_train=[]
y_train=[]
for i in range(0,50):
  for j in range(k,k+45):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=100

In [ ]:
k=45
X_test=[]
y_test=[]
for i in range(0,50):
  for j in range(k,k+5):
    X_test.append(dataset.iloc[j,0])
    y_test.append(dataset.iloc[j,1])
  k+=100

In [ ]:
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
X_train=X_train.iloc[:,:].values
X_test=X_test.iloc[:,:].values
y_train=y_train.iloc[:,:].values
y_test=y_test.iloc[:,:].values

In [ ]:
train=np.concatenate((X_train,y_train),axis=1)
test=np.concatenate((X_test,y_test),axis=1)

In [ ]:
np.random.shuffle(train)
np.random.shuffle(test)

In [ ]:
train=pd.DataFrame(train)
test=pd.DataFrame(test)

In [ ]:
X_train=train.iloc[:,0]
X_test=test.iloc[:,0]
y_train=train.iloc[:,1]
y_test=test.iloc[:,1]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
vectorizer.transform(X_train)
vectorizer.transform(X_test)
tfidf_vector_X = vectorizer.transform(X_train).toarray() 
tfidf_vector_Y = vectorizer.transform(X_test).toarray() 
tfidf_vector_X = tfidf_vector_X[:, :, None] 
tfidf_vector_Y = tfidf_vector_Y[:, :, None]

In [ ]:
a=tfidf_vector_X.shape[0]

b=tfidf_vector_X.shape[1]
tfidf_vector_X=tfidf_vector_X.reshape(a,1,b)
tfidf_vector_Y=tfidf_vector_Y.reshape(tfidf_vector_Y.shape[0],1,b)

In [ ]:
X_train = encode_data(X_train, maxlen, vocab, vocab_size, check)
X_test= encode_data(X_test, maxlen, vocab, vocab_size, check)

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X_train,y_train, test_size = 0.25, random_state = 0)

In [ ]:
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.3
rate_drop_dense = 0.3

In [ ]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x


# define our own softmax function instead of K.softmax
# because K.softmax can not specify axis.
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


# define the margin loss like hinge loss
def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)


class Capsule(Layer):
    """A Capsule Implement with Pure Keras
    There are two vesions of Capsule.
    One is like dense layer (for the fixed-shape input),
    and the other is like timedistributed dense (for various length input).

    The input shape of Capsule must be (batch_size,
                                        input_num_capsule,
                                        input_dim_capsule
                                       )
    and the output shape is (batch_size,
                             num_capsule,
                             dim_capsule
                            )

    Capsule Implement is from https://github.com/bojone/Capsule/
    Capsule Paper: https://arxiv.org/abs/1710.09829
    """

    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):
        """Following the routing algorithm from Hinton's paper,
        but replace b = b + <u,v> with b = <u,v>.

        This change can improve the feature representation of Capsule.

        However, you can replace
            b = K.batch_dot(outputs, hat_inputs, [2, 3])
        with
            b += K.batch_dot(outputs, hat_inputs, [2, 3])
        to realize a standard routing.
        """

        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            o = self.activation(K.batch_dot(c, hat_inputs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(o, hat_inputs, [2, 3])
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)

        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [ ]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3) 
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


In [ ]:
def model2(filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter,cat_output):                                                  # For Character Embedding use this model instead of above model
    d = 300                                                             #Embedding Size
    inputs = Input(shape=(maxlen, vocab_size), name='input', dtype='float32')
    #conv = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[0], activation='relu',input_shape=(maxlen, vocab_size))(inputs)

    conv=KernelConv2D(input_shape=(maxlen ,vocab_size),filters=128,kernel_size=1,kernel_function=PolynomialKernel(p=2, trainable_c=True))(inputs)
    #conv = MaxPooling1D(pool_length=3)(conv)

    conv1 = Convolution1D(filters=nb_filter, kernel_size=filter_kernels[1],
                           activation='relu')(conv)
    #conv1 = MaxPooling1D(pool_length=3)(conv1)

    #conv2 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[2],border_mode='valid', activation='relu')(conv1)
    #conv5 = MaxPooling1D(pool_length=3)(conv2)
    conv6= Capsule(num_capsule=1 ,dim_capsule=72, routings=1,share_weights=True)(conv1)
    conv = Flatten()(conv6)

    #Two dense layers with dropout of .5
    #z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(conv))
    # z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(z))

    pred = Dense(cat_output, activation='softmax', name='output')(conv)

    model = Model(inputs=inputs, outputs=pred)

    #sgd = SGD(lr=0.001, momentum=0.9)
    #model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [ ]:
nb_filter = 500
dense_outputs = 12
filter_kernels = [1,1,1]
cat_output = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model = model2(filter_kernels, dense_outputs,1,b, nb_filter, cat_output)
model.summary()
    

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 140, 70)]         0         
_________________________________________________________________
kernel_conv2d_5 (KernelConv2 (None, 140, 128)          9089      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 137, 500)          256500    
_________________________________________________________________
capsule_5 (Capsule)          (None, 1, 1, 72)          36000     
_________________________________________________________________
flatten_5 (Flatten)          (None, 72)                0         
_________________________________________________________________
output (Dense)               (None, 2)                 146       
Total params: 301,735
Trainable params: 301,735
Non-trainable params: 0
_____________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
model.fit(np.array(X_train), np.array(y_train),validation_data=(np.array(X_eval), np.array(y_eval)),batch_size=32,epochs=100, verbose=1,callbacks=[callback])

Epoch 1/3
46/46 [==============================] - 9s 185ms/step - loss: 0.1939 - accuracy: 0.9483 - val_loss: 0.2334 - val_accuracy: 0.9204
Epoch 2/3
46/46 [==============================] - 9s 186ms/step - loss: 0.1409 - accuracy: 0.9626 - val_loss: 0.1295 - val_accuracy: 0.9633
Epoch 3/3
46/46 [==============================] - 8s 184ms/step - loss: 0.1234 - accuracy: 0.9660 - val_loss: 0.1350 - val_accuracy: 0.9592


In [ ]:
y_pred = model.predict(np.array(tfidf_vector_Y))

y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

y_test=pd.DataFrame(y_test)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):                                                                                                                                                                                                 result.append(j)

predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)

print(result)
print(predicted)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(result,predicted)

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print('Confusion Matrix :')
print(cm)
print('Accuracy Score :',accuracy_score(result, predicted))
print('Report : ')
print(classification_report(result, predicted))